In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout ,GlobalAveragePooling1D

from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/feedback-prize-effectiveness/train.csv')
test = pd.read_csv('../input/feedback-prize-effectiveness/test.csv')
sample_submission = pd.read_csv('../input/feedback-prize-effectiveness/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
train.shape, test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
print(train['discourse_type'].value_counts())
print()
print(train['discourse_effectiveness'].value_counts())

In [ ]:
# Graphs ....

In [ ]:
train['Adequate'] = pd.get_dummies(train['discourse_effectiveness'])['Adequate']
train['Effective'] = pd.get_dummies(train['discourse_effectiveness'])['Ineffective']
train['Ineffective'] = pd.get_dummies(train['discourse_effectiveness'])['Ineffective']

In [ ]:
train.head()

In [ ]:
train['discourse_text'][10]

In [ ]:
len(train['discourse_text'][10])

In [ ]:


def custom_standardization(input_data):
  text = tf.strings.lower(input_data)

  #removing square brackets  
  text = tf.strings.regex_replace(text,'\[.*?\]', '')

  #removing puncuation
  text = tf.strings.regex_replace(text,'[%s]' % re.escape(string.punctuation), '')
  text = tf.strings.regex_replace(text ,'\n' , '')

  #remove words containing numbers
  text = tf.strings.regex_replace(text ,'\w*\d\w*' , '')


  return tf.strings.regex_replace(text,'[%s]' % re.escape(string.punctuation),'')

In [ ]:
custom_standardization(train['discourse_text'][1])

In [ ]:
max_features = 10000 # no of word in vocab
sequence_length = 500

In [ ]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [ ]:
print(np.array(vectorize_layer.get_vocabulary()))
print(len(np.array(vectorize_layer.get_vocabulary())))

In [ ]:
train['discourse_text'].values

In [ ]:
vectorize_layer.adapt(train['discourse_text'].values)

In [ ]:
vectorize_text = vectorize_layer(train['discourse_text'].values)
vectorize_text

In [ ]:
columns_to_br_reomve = ['discourse_id','essay_id','discourse_type','discourse_effectiveness']
train = train.drop(columns_to_br_reomve, axis=1)

In [ ]:
train.head()

In [ ]:
y = train.drop(['discourse_text'],axis=1)
X = vectorize_text

In [ ]:
embedding_dim = 64

model = tf.keras.Sequential([
  Embedding(max_features + 1, embedding_dim),
  Dropout(0.2),
  GlobalAveragePooling1D(),
  Dropout(0.2),
  Dense(3, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X,y,epochs=3,verbose=1)

In [ ]:
x_test = vectorize_layer(test['discourse_text'].values)
Predict = model.predict(x_test)
Predict

In [ ]:
prediction = pd.DataFrame(Predict, columns=["Ineffective", "Adequate", "Effective"])

In [ ]:
prediction["discourse_id"] = sample_submission["discourse_id"] 
titles = ['discourse_id','Ineffective', 'Adequate', 'Effective']
prediction = prediction.reindex(columns = titles)
prediction

In [ ]:
submission = pd.DataFrame(prediction)
submission.to_csv('submission.csv', index = False)
print("My competition submission: \n\n", submission)